# Text Classification

* 문장, 문단 또는 글을 어떤 카테고리에 분류하는 작업을 텍스트 분류라고 합니다.
* 텍스트 분류는 지도학습입니다.
* Input: 하나의 문장, 문단 혹은 문서
* Output: 유한한 C 개의 카테고리

### 예시
* 감성 분석
* 카테고리 분류
* 의도 분류

# 문자을 어떻게 처리할까?

* 문장은 일련의 토큰(tokens)으로 구성되어 있다. Text tokens 은 주관적, 임의적인 성격을 띄고 있다.
* 토큰을 나누는 기준은 다양하다.
   - 공백(White Spaces)
   - 형태소(Morphs)
   - 어절
   - 비트 숫자
* 컴퓨터에게 단어를 숫자로 표현하기 위해서, 단어장(Vocabulary)을 만들고, 중복되지 않는 인덱스(index) 로 바꾼다.
* 궁극적으로 모든 문장을 일련의 정수로 바꿔준다. 이를 인코딩(Encoding)이라고 한다.
* 하지만 관계없는 숫자의 나열로 인코딩하는 것은 우리가 원하는 것이 아니다. 여전히 주관적인 숫자들 뿐이다.
* 우리는 비슷한 의미의 단어는 같이 있고, 아니면 멀리 떨어져 있는 관계를 만들고 싶다. 그렇다면 어떻게 관계를 만들어 줘야 할까?

# 문자처리 예시
* "커넥트 제단에서 강의 중 입니다."
* /커넥트/제단/에서/강의/중/입니다/./
* 사전에서 다음 단어들에 해당하는 Index(정수값)을 찾아 변환한다.
* 5241, 827, 20, 288, 12, 19, 5

# One Hot Encoding

* 길이가 단어장의 총 길이(∣V∣)인 벡터에서, 단어의 index 위치에 있는 값은 1, 나머지는 0으로 구성한다.

* x=[0,0,0,⋯,0,1,0,⋯,0,0]∈{0,1}
​∣V∣
​​  
* 단점: 모든 토큰 간에 거리가 같다. 하지만 모든 단어의 뜻이 같지 않기 때문에 거리가 달라져야 우리가 원하는 단어간의 관계가 성립 된다. 

# 어떻게 신경망이 토큰의 의미를 잡아낼 수 있을까?
* 결론은 각 토큰을 연속 벡터 공간(Continuous vector space) 에 투영하는 방법이다. 이를 임베딩(Embedding) 이라고도 한다.
* Table Look Up: 각 one hot encoding 된 토큰에게 벡터를 부여하는 과정이다. 
* 실질적으로 one hot encoding 벡터( x )와 연속 벡터 공간( W )을 내적 한 것이다.
* Table Look Up 과정을 거친후 모든 문장 토큰은 연속적이고 높은 차원의 벡터로 변한다.
* X=(e
​1
​​ ,e
​2
​​ ,⋯,e
​T
​​ )where e
​t
​​ ∈R
​d
​​  

# Continuous bag-of words (CBoW)
* 단어장을 단어 주머니로 보게 되고, 이에 따라 단어의 순서는 무시한다.
* 문장에 대한 표현은 단어 벡터들을 평균시킨 벡터로 구한다.
* 효과가 좋기 때문에 가장 먼저 시도해야한다. (Baseline Model)
* 공간상에서 가까우면 비슷한 의미, 아니면 멀리 떨어져 있는 것이다.

# Relation Network(Skip-Bigram)
* 문장안에 있는 모든 토큰 쌍(pairs)을 보고, 각 쌍에 대해서 신경망을 만들어서 문장표현을 찾는다.
* 장점 : 여러 단어로 된 표현을 탐지할 수 있다.
* 단점 : 모든 단어간의 관계를 보기 때문에, 전혀 연관이 없는 단어도 보게 된다.

# Convolution Neural Network (CNN)
* K-gram을 계측적으로 보게 된다. (hierachically)
* Layer를 쌓을 때 마다, 점진적으로 넓은 범위를 보기 때문에 "단어 > 다중 단어 표현 > 구절 > 문장" 순으로 보는 인간의 인식과도 알맞다.
* 1차원의 Convolution Network이다.
* 장점 : 좁은 지역간 단어의 관계를 볼 수 있다.

# RN과 CNN과의 관계

#### RN
* 모든 다른 토큰의 관게를 본다. 모든 단어간의 관계를 봐서 효율적이지 못하다.
* t번째와 각각 다른 토큰들과의 관계
* $ h_t = f(x_t,x_1) + ... + f(x_t,x_{t-1}) + f(x_t,x_{t+1}) + ... + f(x_t,x_T) $

#### CNN
* 작은 범위의 토큰 관계를 본다. 따라서 더 먼 거리의 단어간의 관계가 있을 경우 탐지할 수 없거나 더 많은 convolution 층이 필요하다.
* t번째와 그 주변과의 관계
* $ h_t = f(x_t,x_{t-k}) + ... + f(x_t,x_t) + ... + f(x_t,x_{t+k}) $

#### 하지만, CNN 방식을 가중치가 부여된 RN의 일종으로 볼 수도 있다.
* $ h_t = \Sigma_{t'=1}^{T} I(|t'-t|<= k)f(x_t,x_{t'})$
* $ I(S) = $ 1 if S is True,    0 otherwise

#### 그렇다면 가중치가 0과 1이 아닌 그사의 값으로 계산할 수 있다면...?

# Self Attention
* $ h_t = \Sigma_{t'=1}^{T} \alpha(x_t,x_{t'})f(x_t,x_{t'})$


* $ \alpha = {exp(\beta(x_t,x_{t'})) \over \Sigma_{t'=1}^{T} exp(\beta(x_t,x_{t'}))} $


* $ where : \beta(x_t,x_{t'})=RN(x_t,x_{t'})$


* 장점
  * Long range & short range dependency 극복할 수 있다.
  * 관계가 낮은 토큰은 억제하고 관계가 높은 토큰은 강조할 수 있다.
* 단점
  * 계산 복잡도가 높고 counting 같은 특정 연산이 쉽지 않다.

# Recurrent Neural Network (RNN)
* 메모리를 가지고 있어서 현재까지 읽는 정보는 저장할 수 있다.
* 문장의 정보를 시간의 순서에 따라 압축할 수 있다.

* 단점
  * 문장이 많이 길어질수록 고정된 메모리에 압축된 정보를 담아야 하기 때문에 앞에서 학습한 정보를 잊는다. 이는 곧 정보의 손실을 의미한다.
  * 토큰을 순차적으로 하나씩 읽어야 하기 때문에, 훈련할 때 속도가 기타 네트워크보다 느리다.
 
* Long Term Dependency 해결방법
  * bidirectional network를 쓴다. (양쪽 방향에서 데이터를 읽어온다.)
  * LSTM, GRU 등 RNN의 변형을 사용한다.

# Text Classification의 방법
* Continuous bag-of words (CBoW)
* Relation Network(Skip-Bigram)
* Convolution Neural Network (CNN)
* Self Attention
* Recurrent Neural Network (RNN)